In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine 
import numpy as np

In [2]:
#Read in the Crime Excel File
crime_file = "Crime Rates by Metropolitan Areas 2016.xls"
crime_df = pd.read_excel(crime_file)
crime_df.head()

,"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.",Note that not all areas show Violent Crime Totals.,Population,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.","Rate per 100,000 inhabitants",2157729,245.3,23.4,4.9,123.0,94,1190.4,255.6,778.7,156.1
1,"Fairbanks, AK M.S.A.","Rate per 100,000 inhabitants",34657,669.4,20.2,101.0,92.3,455.9,4377.2,499.2,3482.7,395.3
2,"Detroit-Dearborn-Livonia, MI M.D.","Rate per 100,000 inhabitants",1747582,1034.2,19.4,63.2,220.9,730.7,3132.6,735.0,1725.4,672.1
3,"New Orleans-Metairie, LA M.S.A.","Rate per 100,000 inhabitants",1272995,558.8,18.9,56.0,162.0,321.9,3001.3,468.3,2245.5,287.5
4,"Memphis, TN-MS-AR M.S.A.","Rate per 100,000 inhabitants",1346931,1082.1,18.5,54.6,269.4,739.6,3868.2,934.3,2623.2,310.6


In [3]:
#Read in the Weather Excel File
weather_file = "cloudy_updated_new.csv"
weather_pd = pd.read_csv(weather_file)
weather_pd.head()

,Station,Location,Year,CL_Jan,PC_Jan,CD_Jan,CL_Feb,PC_Feb,CD_Feb,CL_Mar,...,CD_Oct,CL_Nov,PC_Nov,CD_Nov,CL_Dec,PC_Dec,CD_Dec,CL_Annual,PC_Annual,CD_Annual
0,13876,"BIRMINGHAM ,AL",37,7,6,18,7,6,15,7,...,9,10,7,13,8,7,16,99,111,155
1,3856,"HUNTSVILLE, AL",27,7,6,18,7,6,16,7,...,12,9,7,14,8,6,18,100,101,164
2,13894,"MOBILE, AL",47,8,6,17,8,7,14,9,...,9,11,7,12,9,6,16,102,116,147
3,13895,"MONTGOMERY, AL",51,7,6,17,8,6,14,8,...,10,11,6,13,9,6,16,107,107,151
4,26451,"ANCHORAGE, AK",44,7,5,19,6,4,18,8,...,21,6,5,20,6,4,21,61,65,239


In [4]:
#Transform Crime Data Frame 
crime_cols = crime_df[["Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.","Aggravated\nassault", "Violent\ncrime","Murder and\nnonnegligent\nmanslaughter", "Robbery"]]
crime_cols.head()

,"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.",Aggravated assault,Violent crime,Murder and nonnegligent manslaughter,Robbery
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4


In [5]:
#Rename the Columns
crime_clean = crime_cols.rename(columns={"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.": "Location",
                                        "Aggravated\nassault": "Aggravated Assault",
                                        "Murder and\nnonnegligent\nmanslaughter": "Murder",
                                        "Violent\ncrime": "Violent Crime"})
crime_clean.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4


In [6]:
#Drop empty data
crime_clean.dropna(inplace = True)
crime_clean=pd.concat([crime_clean,crime_clean.Location.str.split(',',expand=True)],1)

crime_clean.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,0,1,2
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,None


In [7]:
#Rename the new columns
crime_split = crime_clean.rename(columns={0: "City1",
                                        1: "State"})
crime_split.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State,2
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,None


In [8]:
#Drop the columns I dont need
crime_split = crime_split[["Location", "Aggravated Assault", "Violent Crime", "Murder", "Robbery", "City1", "State"]]
crime_split.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.


In [9]:
#Strip down the city names 
crime_split2=pd.concat([crime_split,crime_split.City1.str.split('-',expand=True)],1)
crime_split2.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State,0,1,2,3
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,San Juan,Carolina,Caguas,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,Fairbanks,None,None,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,Detroit,Dearborn,Livonia,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,New Orleans,Metairie,None,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,Memphis,None,None,None


In [10]:
#Clean up the final table (haha..what we thought was the final table)
crime_split2 = crime_split2.rename(columns={0: "City"})

crime_final2 = crime_split2[["State", "Aggravated Assault", "Violent Crime", "Murder", "Robbery", "City"]]

In [11]:
#Sorting the Cities by Alphabetical Order
crime_final3 = crime_final2.sort_values(by=['City'])

In [12]:
# Applying upper() method on 'City' column 
crime_final3['City'] = crime_final3['City'].str.upper()

In [13]:
# Finding the Rows that have Puerto Rico
crime_final3[crime_final3['State'].str.contains('Puerto')]

,State,Aggravated Assault,Violent Crime,Murder,Robbery,City
0,Puerto Rico M.S.A.,94,245.3,23.4,123.0,SAN JUAN


In [14]:
#Drop Puetro Rico
crime_final3.drop([0], inplace=True)

In [15]:
#Strip down the city names 
crime_final4=pd.concat([crime_final3,crime_final3.State.str.split(' ',expand=True)],1)
crime_final4.head()

,State,Aggravated Assault,Violent Crime,Murder,Robbery,City,0,1,2
94,TX M.S.A.,227.8,361.4,6.5,80.6,ABILENE,,TX,M.S.A.
108,OH M.S.A.2,121,253.1,6.0,77.3,AKRON,,OH,M.S.A.2
367,OR M.S.A.,59,113.9,0.0,27.0,ALBANY,,OR,M.S.A.
282,NY M.S.A.,180,289.7,2.0,68.7,ALBANY,,NY,M.S.A.
14,GA M.S.A.,534.9,734.8,11.8,159.9,ALBANY,,GA,M.S.A.


In [16]:
#Merging City and State
crime_final4['city_state'] = crime_final4['City'] + ', ' + crime_final4[1]


In [17]:
#Dropping the Extra States off of the end
crime_final4[crime_final4['city_state'].str.contains('-')]

,State,Aggravated Assault,Violent Crime,Murder,Robbery,City,0,1,2,city_state
68,GA-SC M.S.A.,214.9,342.8,7.4,84.5,AUGUSTA,,GA-SC,M.S.A.,"AUGUSTA, GA-SC"
102,MO-IL M.S.A.,379,480.4,6.1,73.7,CAPE GIRARDEAU,,MO-IL,M.S.A.,"CAPE GIRARDEAU, MO-IL"
109,TN-GA M.S.A.,404.8,540.8,6.0,91.8,CHATTANOOGA,,TN-GA,M.S.A.,"CHATTANOOGA, TN-GA"
25,IL-IN-WI M.S.A.2,232.8,443,10.2,166.8,CHICAGO,,IL-IN-WI,M.S.A.2,"CHICAGO, IL-IN-WI"
157,OH-KY-IN M.S.A.,109.2,257.3,4.7,102.3,CINCINNATI,,OH-KY-IN,M.S.A.,"CINCINNATI, OH-KY-IN"
88,TN-KY M.S.A.,309.7,427.6,6.7,70.9,CLARKSVILLE,,TN-KY,M.S.A.,"CLARKSVILLE, TN-KY"
23,GA-AL M.S.A.,279.2,518.3,10.4,189.9,COLUMBUS,,GA-AL,M.S.A.,"COLUMBUS, GA-AL"
103,MD-WV M.S.A.,213.7,318,6.1,72.9,CUMBERLAND,,MD-WV,M.S.A.,"CUMBERLAND, MD-WV"
218,IA-IL M.S.A.4,278.8,393.2,3.1,62.7,DAVENPORT,,IA-IL,M.S.A.4,"DAVENPORT, IA-IL"
346,MN-WI M.S.A.,150.8,229.9,1.1,32.9,DULUTH,,MN-WI,M.S.A.,"DULUTH, MN-WI"


In [18]:
crime_final4=crime_final4[["Aggravated Assault", "Violent Crime", "Murder", "Robbery", "city_state"]]

In [19]:
#Strip down the city names 
crime_final5=pd.concat([crime_final4,crime_final4.city_state.str.split('-',expand=True)],1)
crime_final5.head()

,Aggravated Assault,Violent Crime,Murder,Robbery,city_state,0,1,2,3
94,227.8,361.4,6.5,80.6,"ABILENE, TX","ABILENE, TX",None,None,None
108,121,253.1,6.0,77.3,"AKRON, OH","AKRON, OH",None,None,None
367,59,113.9,0.0,27.0,"ALBANY, OR","ALBANY, OR",None,None,None
282,180,289.7,2.0,68.7,"ALBANY, NY","ALBANY, NY",None,None,None
14,534.9,734.8,11.8,159.9,"ALBANY, GA","ALBANY, GA",None,None,None


In [20]:
crime_final5=crime_final5[["Aggravated Assault", "Violent Crime", "Murder", "Robbery", 0]]
crime_final5.head()

,Aggravated Assault,Violent Crime,Murder,Robbery,0
94,227.8,361.4,6.5,80.6,"ABILENE, TX"
108,121,253.1,6.0,77.3,"AKRON, OH"
367,59,113.9,0.0,27.0,"ALBANY, OR"
282,180,289.7,2.0,68.7,"ALBANY, NY"
14,534.9,734.8,11.8,159.9,"ALBANY, GA"


In [21]:
#Rename the new columns FINAL CRIME DF
crime_final6 = crime_final5.rename(columns={0: "City_State"})
crime_final6.head()

,Aggravated Assault,Violent Crime,Murder,Robbery,City_State
94,227.8,361.4,6.5,80.6,"ABILENE, TX"
108,121,253.1,6.0,77.3,"AKRON, OH"
367,59,113.9,0.0,27.0,"ALBANY, OR"
282,180,289.7,2.0,68.7,"ALBANY, NY"
14,534.9,734.8,11.8,159.9,"ALBANY, GA"


In [22]:
# The start of the weather data
#Using concat to break apart "Location" into 2 columns and splitting by comma
weather_pd=pd.concat([weather_pd,weather_pd.Location.str.split(',',expand=True)],1)
weather_pd.head(360)

,Station,Location,Year,CL_Jan,PC_Jan,CD_Jan,CL_Feb,PC_Feb,CD_Feb,CL_Mar,...,PC_Nov,CD_Nov,CL_Dec,PC_Dec,CD_Dec,CL_Annual,PC_Annual,CD_Annual,0,1
0,13876,"BIRMINGHAM ,AL",37,7,6,18,7,6,15,7,...,7,13,8,7,16,99,111,155,BIRMINGHAM,AL
1,3856,"HUNTSVILLE, AL",27,7,6,18,7,6,16,7,...,7,14,8,6,18,100,101,164,HUNTSVILLE,AL
2,13894,"MOBILE, AL",47,8,6,17,8,7,14,9,...,7,12,9,6,16,102,116,147,MOBILE,AL
3,13895,"MONTGOMERY, AL",51,7,6,17,8,6,14,8,...,6,13,9,6,16,107,107,151,MONTGOMERY,AL
4,26451,"ANCHORAGE, AK",44,7,5,19,6,4,18,8,...,5,20,6,4,21,61,65,239,ANCHORAGE,AK
5,25308,"ANNETTE, AK",41,4,4,23,3,4,21,4,...,3,24,3,4,24,40,60,265,ANNETTE,AK
6,27502,"BARROW, AK",54,6,2,3,12,6,11,14,...,3,11,4,0,0,66,52,187,BARROW,AK
7,26615,"BETHEL, AK",34,8,5,18,10,4,14,9,...,5,19,8,5,18,62,65,239,BETHEL,AK
8,26533,"BETTLES,AK",5,15,5,11,10,4,14,9,...,6,14,9,7,15,86,93,187,BETTLES,AK
9,26415,"BIG DELTA,AK",3,14,8,9,5,7,17,8,...,6,17,7,7,17,60,93,212,BIG DELTA,AK


In [23]:
#Renaming new columns
weather_pd = weather_pd.rename(columns={0: "City", 1: "State"})
weather_pd.head()

,Station,Location,Year,CL_Jan,PC_Jan,CD_Jan,CL_Feb,PC_Feb,CD_Feb,CL_Mar,...,PC_Nov,CD_Nov,CL_Dec,PC_Dec,CD_Dec,CL_Annual,PC_Annual,CD_Annual,City,State
0,13876,"BIRMINGHAM ,AL",37,7,6,18,7,6,15,7,...,7,13,8,7,16,99,111,155,BIRMINGHAM,AL
1,3856,"HUNTSVILLE, AL",27,7,6,18,7,6,16,7,...,7,14,8,6,18,100,101,164,HUNTSVILLE,AL
2,13894,"MOBILE, AL",47,8,6,17,8,7,14,9,...,7,12,9,6,16,102,116,147,MOBILE,AL
3,13895,"MONTGOMERY, AL",51,7,6,17,8,6,14,8,...,6,13,9,6,16,107,107,151,MONTGOMERY,AL
4,26451,"ANCHORAGE, AK",44,7,5,19,6,4,18,8,...,5,20,6,4,21,61,65,239,ANCHORAGE,AK


In [24]:
#Checking to see unique values in each column
#weather_pd.CL_Jan.unique()

In [25]:
# Checking the names of all of the columns
weather_pd.columns

Index(['Station', 'Location', 'Year', 'CL_Jan', 'PC_Jan', 'CD_Jan', 'CL_Feb',
       'PC_Feb', 'CD_Feb', 'CL_Mar', 'PC_Mar', 'CD_Mar', 'CL_Apr', 'PC_Apr',
       'CD_Apr', 'CL_May', 'PC_May', 'CD_May', 'CL_Jun', 'PC_Jun', 'CD_Jun',
       'CL_Jul', 'PC_Jul', 'CD_Jul', 'CL_Aug', 'PC_Aug', 'CD_Aug', 'CL_Sep',
       'PC_Sep', 'CD_Sep', 'CL_Oct', 'PC_Oct', 'CD_Oct', 'CL_Nov', 'PC_Nov',
       'CD_Nov', 'CL_Dec', 'PC_Dec', 'CD_Dec', 'CL_Annual', 'PC_Annual',
       'CD_Annual', 'City', 'State'],
      dtype='object')

In [26]:
# Checking to see if every column has the same amount of entries
weather_pd.count()

Station      270
Location     270
Year         270
CL_Jan       270
PC_Jan       270
CD_Jan       270
CL_Feb       270
PC_Feb       270
CD_Feb       270
CL_Mar       270
PC_Mar       270
CD_Mar       270
CL_Apr       270
PC_Apr       270
CD_Apr       270
CL_May       270
PC_May       270
CD_May       270
CL_Jun       270
PC_Jun       270
CD_Jun       270
CL_Jul       270
PC_Jul       270
CD_Jul       270
CL_Aug       270
PC_Aug       270
CD_Aug       270
CL_Sep       270
PC_Sep       270
CD_Sep       270
CL_Oct       270
PC_Oct       270
CD_Oct       270
CL_Nov       270
PC_Nov       270
CD_Nov       270
CL_Dec       270
PC_Dec       270
CD_Dec       270
CL_Annual    270
PC_Annual    270
CD_Annual    270
City         270
State        270
dtype: int64

In [27]:
#Checking to see the column types
weather_pd.dtypes

Station       int64
Location     object
Year         object
CL_Jan       object
PC_Jan        int64
CD_Jan        int64
CL_Feb       object
PC_Feb        int64
CD_Feb        int64
CL_Mar       object
PC_Mar        int64
CD_Mar        int64
CL_Apr       object
PC_Apr        int64
CD_Apr        int64
CL_May       object
PC_May        int64
CD_May        int64
CL_Jun       object
PC_Jun        int64
CD_Jun        int64
CL_Jul       object
PC_Jul        int64
CD_Jul        int64
CL_Aug       object
PC_Aug        int64
CD_Aug        int64
CL_Sep       object
PC_Sep        int64
CD_Sep        int64
CL_Oct       object
PC_Oct        int64
CD_Oct        int64
CL_Nov       object
PC_Nov        int64
CD_Nov        int64
CL_Dec       object
PC_Dec        int64
CD_Dec        int64
CL_Annual     int64
PC_Annual     int64
CD_Annual     int64
City         object
State        object
dtype: object

In [28]:
#Checking to see unique values in each column
weather_pd.CL_Jan.unique()

array(['7', '8', '4', '6', '15', '14', '2', '9', '5', '11', '10', '12',
       '13', '3', '1', '*', '0'], dtype=object)

In [29]:
#Checking to see unique values in each column
weather_pd.CL_Feb.unique()

array(['7', '8', '6', '3', '12', '10', '5', '2', '4', '9', '11', '13',
       '15', '14', '*', '1', '0'], dtype=object)

In [30]:
#Checking to see unique values in each column
weather_pd.CL_Mar.unique()

array(['7', '9', '8', '4', '14', '2', '10', '6', '12', '11', '3', '5',
       '15', '13', '17', '1', '*'], dtype=object)

In [31]:
#Checking to see unique values in each column
weather_pd.CL_Apr.unique()

array(['9', '10', '6', '3', '11', '7', '2', '1', '5', '4', '12', '17',
       '15', '21', '14', '8', '13', '16', '*'], dtype=object)

In [32]:
#Checking to see unique values in each column
weather_pd.CL_May.unique()

array(['8', '9', '4', '3', '7', '2', '*', '5', '1', '6', '15', '21', '20',
       '17', '24', '18', '12', '10', '11', '14', '13', '19'], dtype=object)

In [33]:
#Checking to see unique values in each column
weather_pd.CL_Jun.unique()

array(['7', '8', '3', '2', '4', '*', '1', '6', '5', '18', '23', '21',
       '20', '25', '10', '9', '12', '14', '16', '17', '22', '15', '11',
       '13'], dtype=object)

In [34]:
#Checking to see unique values in each column
weather_pd.CL_Jul.unique()

array(['5', '7', '4', '6', '3', '2', '*', '1', '9', '17', '10', '11',
       '20', '12', '26', '22', '27', '18', '13', '21', '25', '14', '8',
       '15', '19', '16', '23', '0'], dtype=object)

In [35]:
#Checking to see unique values in each column
weather_pd.CL_Aug.unique()

array(['7', '9', '6', '8', '3', '4', '1', '2', '*', '5', '10', '18', '12',
       '22', '26', '23', '25', '19', '14', '24', '15', '17', '11', '13',
       '16', '20', '21', '0'], dtype=object)

In [36]:
#Checking to see unique values in each column
weather_pd.CL_Sep.unique()

array(['9', '10', '4', '1', '2', '7', '5', '*', '6', '3', '16', '22',
       '19', '18', '24', '12', '11', '23', '15', '13', '21', '14', '8',
       '17'], dtype=object)

In [37]:
#Checking to see unique values in each column
weather_pd.CL_Oct.unique()

array(['14', '12', '5', '2', '4', '1', '3', '6', '8', '7', '17', '20',
       '19', '23', '13', '16', '18', '15', '11', '9', '10', '*', '0'],
      dtype=object)

In [38]:
#Checking to see unique values in each column
weather_pd.CL_Nov.unique()

array(['10', '9', '11', '6', '3', '7', '1', '5', '4', '8', '15', '18',
       '16', '19', '12', '13', '14', '2', '*', '0'], dtype=object)

In [39]:
#Checking to see unique values in each column
weather_pd.CL_Nov.unique()

array(['10', '9', '11', '6', '3', '7', '1', '5', '4', '8', '15', '18',
       '16', '19', '12', '13', '14', '2', '*', '0'], dtype=object)

In [40]:
# Checking to see what rows have * instead of numbers
weather_pd.loc[weather_pd["CL_Jan"]=="*"]
weather_pd.loc[weather_pd["CL_Feb"]=="*"]
weather_pd.loc[weather_pd["CL_Mar"]=="*"]
weather_pd.loc[weather_pd["CL_Apr"]=="*"]
weather_pd.loc[weather_pd["CL_May"]=="*"]
weather_pd.loc[weather_pd["CL_Jun"]=="*"]
weather_pd.loc[weather_pd["CL_Jul"]=="*"]
weather_pd.loc[weather_pd["CL_Aug"]=="*"]
weather_pd.loc[weather_pd["CL_Sep"]=="*"]
weather_pd.loc[weather_pd["CL_Oct"]=="*"]
weather_pd.loc[weather_pd["CL_Nov"]=="*"]
weather_pd.loc[weather_pd["CL_Dec"]=="*"]

,Station,Location,Year,CL_Jan,PC_Jan,CD_Jan,CL_Feb,PC_Feb,CD_Feb,CL_Mar,...,PC_Nov,CD_Nov,CL_Dec,PC_Dec,CD_Dec,CL_Annual,PC_Annual,CD_Annual,City,State
261,40309,"KOROR, PC",53,*,7,23,*,7,21,1,...,8,22,*,7,24,4,86,273,KOROR,PC
265,40504,"POHNPEI- CAROLINE IS., PC",53,*,4,26,*,4,24,*,...,6,24,*,5,26,2,64,298,POHNPEI- CAROLINE IS.,PC
266,40505,"CHUUK- E. CAROLINE IS., P",56,*,4,27,*,3,25,*,...,4,26,*,3,28,0,45,318,CHUUK- E. CAROLINE IS.,P


In [41]:
# Dropping the rows that contain asteric
weather_pd.drop([9,10,258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269], inplace=True)

In [42]:
weather_pd["CL_Annual_Check"] = pd.to_numeric(weather_pd["CL_Jan"]) + pd.to_numeric(weather_pd["CL_Feb"]) + pd.to_numeric(weather_pd["CL_Mar"]) + pd.to_numeric(weather_pd["CL_Apr"]) + pd.to_numeric(weather_pd["CL_May"]) + pd.to_numeric(weather_pd["CL_Jun"]) + pd.to_numeric(weather_pd["CL_Jul"]) + pd.to_numeric(weather_pd["CL_Aug"]) + pd.to_numeric(weather_pd["CL_Sep"]) + pd.to_numeric(weather_pd["CL_Oct"]) + pd.to_numeric(weather_pd["CL_Nov"]) + pd.to_numeric(weather_pd["CL_Dec"])


In [43]:
#Checking to see whether two columns are equal - since theyre not equal, will go ahead and use CL_annual number
weather_pd['CL_Annual_Check'].equals(weather_pd['CL_Annual'])

False

In [44]:
#Only using columns that we need
weather_updated = weather_pd[["City", "State","CL_Annual"]]

In [45]:
weather_updated2 = weather_updated.sort_values(by=['City'])

In [46]:
weather_updated2['City_State'] = weather_updated2['City'] + ',' + weather_updated2['State']
weather_updated2.head()


,City,State,CL_Annual,City_State
207,ABERDEEN,SD,101,"ABERDEEN, SD"
217,ABILENE,TX,149,"ABILENE, TX"
176,AKRON,OH,68,"AKRON, OH"
47,ALAMOSA,CO,148,"ALAMOSA, CO"
159,ALBANY,NY,69,"ALBANY, NY"


In [47]:
#Only using columns that we need
weather_final = weather_updated2[["City_State","CL_Annual"]]

In [48]:
merge_table_final = pd.merge(crime_final6, weather_final, on="City_State", how="inner")
merge_table_final

,Aggravated Assault,Violent Crime,Murder,Robbery,City_State,CL_Annual
0,227.8,361.4,6.5,80.6,"ABILENE, TX",149
1,121,253.1,6.0,77.3,"AKRON, OH",68
2,180,289.7,2.0,68.7,"ALBANY, NY",69
3,596.7,901,8.3,238.7,"ALBUQUERQUE, NM",167
4,413.7,592.7,5.3,102.2,"AMARILLO, TX",157
5,707.8,1114.9,8.9,224.6,"ANCHORAGE, AK",61
6,158.9,240.4,5.1,54.7,"ASHEVILLE, NC",99
7,214.6,341.5,2.9,85.9,"ATHENS, GA",113
8,228.2,403.2,6.8,144.1,"ATLANTA, GA",110
9,187.4,316.9,3.4,68.7,"AUSTIN, TX",115


In [49]:
merge_table_final.count()

Aggravated Assault    137
Violent Crime         137
Murder                137
Robbery               137
City_State            137
CL_Annual             137
dtype: int64

In [50]:
merge_table_final.head(137)

,Aggravated Assault,Violent Crime,Murder,Robbery,City_State,CL_Annual
0,227.8,361.4,6.5,80.6,"ABILENE, TX",149
1,121,253.1,6.0,77.3,"AKRON, OH",68
2,180,289.7,2.0,68.7,"ALBANY, NY",69
3,596.7,901,8.3,238.7,"ALBUQUERQUE, NM",167
4,413.7,592.7,5.3,102.2,"AMARILLO, TX",157
5,707.8,1114.9,8.9,224.6,"ANCHORAGE, AK",61
6,158.9,240.4,5.1,54.7,"ASHEVILLE, NC",99
7,214.6,341.5,2.9,85.9,"ATHENS, GA",113
8,228.2,403.2,6.8,144.1,"ATLANTA, GA",110
9,187.4,316.9,3.4,68.7,"AUSTIN, TX",115


In [51]:
merge_table_final.dropna(inplace = True)

In [52]:
merge_table_final.count()

Aggravated Assault    137
Violent Crime         137
Murder                137
Robbery               137
City_State            137
CL_Annual             137
dtype: int64

In [ ]:
connection_string = "root:<abcdefg12345>@3306/weathercrime_db"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
# Confirm table
engine.table_names()

In [ ]:
merge_table_final.to_sql(name='wcjoin', con=engine, if_exists='append', index=True)